# Riproduzione modelli e addestramento

In [1]:
import pandas as pd 
import numpy as np
import os
dataset_folder = 'dataset_new'

In [2]:
sensors = ['sensor2', 'sensor3', 'sensor4','sensor7','sensor8','sensor9','sensor11','sensor12','sensor13','sensor14',
           'sensor15','sensor17','sensor20', 'sensor21']

In [3]:
train = []
files = os.listdir(dataset_folder)
train_files = [file for file in files if file.startswith('scaled')]

for file in files: 
    path_to_file = dataset_folder + '/'+file
    df = pd.read_csv(path_to_file, index_col=0) #index_col = 0 perché sennò viene aggiunta la colonna unnamed. 
    train.append(df)

train[0].head() 


,unit_number,time_cycle,op_setting1,op_setting2,op_setting3,RUL,sensor2,sensor3,sensor4,sensor6,...,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
0,1.0,1.0,-0.0005,0.0004,100.0,258.0,0.355972,0.370523,0.308580,1.0,...,0.623529,0.204233,0.348571,0.231279,0.642857,0.239116,0.647755,0.272727,0.559524,0.446331
1,1.0,2.0,0.0008,-0.0003,100.0,257.0,0.388759,0.399100,0.309360,1.0,...,0.647059,0.202431,0.308571,0.236882,0.654762,0.278567,0.685659,0.363636,0.488095,0.534836
2,1.0,3.0,-0.0014,-0.0002,100.0,256.0,0.313817,0.353298,0.445398,1.0,...,0.664706,0.241484,0.302857,0.217015,0.636905,0.264526,0.564462,0.272727,0.404762,0.458577
3,1.0,4.0,-0.0020,0.0001,100.0,255.0,0.487119,0.417107,0.237285,1.0,...,0.647059,0.215326,0.314286,0.240448,0.684524,0.245612,0.558909,0.363636,0.470238,0.391966
4,1.0,5.0,0.0016,0.0000,100.0,254.0,0.196721,0.476218,0.321217,1.0,...,0.670588,0.268799,0.262857,0.245033,0.654762,0.252109,0.556736,0.363636,0.577381,0.543371


Proviamo, al contrario di quanto fatto in precedenza, ad addestrare le reti in maniera separata sulle 4 componenti del dataframe e poi confrontare i dati 

In [4]:
##Preparo i dati di FD0001 per l'input. 
sequence_length = 30 #Rappresenta i Time Steps del layer di input

from keras.utils import pad_sequences

columns_to_keep = sensors 
columns_to_keep.append('RUL')
df = train[0]
selected_columns = df[sensors]
dataset_array = selected_columns.values



sequences = [dataset_array[i:i+sequence_length] for i in range(len(dataset_array) - sequence_length + 1)]

sequences_padded = pad_sequences(sequences, maxlen=sequence_length, dtype='float32', padding='post')

X = sequences_padded[:, :, :-1]
y = sequences_padded[:, :, -1]


2023-09-18 15:18:24.440918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 15:18:25.403741: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/lib64:/home/aliquodfahriam/miniconda3/envs/tensorflowEnv/lib/
2023-09-18 15:18:25.403889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/

In [5]:
print(X.shape, y.shape)

(24691, 30, 14) (24691, 30)


In [6]:
#Splitting del dataset
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from keras import backend as K 
import tensorflow as tf 
import math 
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


# def custom_score(y_true, y_pred):
#      sum = 0 
#      for true, predicted in zip(y_true, y_pred): 
#           d_i = predicted - true
#           if d_i < 0:
#              esponente = 1/(d_i/13)
#              sum += (math.e ** esponente) -1 
#           else: 
#              esponente =  d_i/10
#              sum += (math.e ** esponente) -1 
     
#      return sum 

@tf.function
def custom_score2(y_true, y_pred):
    d_i = y_pred - y_true
    esponente = tf.where(d_i < 0, 1.0 / (d_i / 13.0), d_i / 10.0)
    sum = tf.reduce_sum(tf.exp(esponente) - 1.0)
    
    return sum


# #Possibili valori di alfa = [0.2, 0.4]
# def custom_loss(y_true, y_pred):
#        alpha = 0.2 
#        difference = y_pred - y_true
#        if difference < 0: 
#              loss = 2 *alpha *difference
#              return loss
#        else: 
#              loss = 2*(alpha + (1 - 2*alpha))*difference
#              return loss
       


@tf.function
def custom_loss2(y_true, y_pred):
    alpha = 0.2
    difference = y_pred - y_true
    squared_difference = tf.square(y_pred - y_true)
    
    # Calcola la loss per ciascun elemento
    loss = tf.where(difference < 0, 2 * alpha * squared_difference, 2 * (alpha + (1 - 2 * alpha)) * squared_difference)
    
    # Calcola la media delle loss
    return tf.reduce_mean(loss)

      

In [8]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam

#Replica della rete neurale proposta come "small LSTM"
def createLSTMsmallModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, input_shape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))    
    flatten = Flatten()
    dense1 = Dense(d1Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    # output layer
    outL = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    #layers
    layers = [lstm1, lstm2,flatten,  dense1, dense2, outL]
    model = Sequential(layers)
    #Abbiamo aggiunto le nostre funzioni di loss e accuracy definite precedentemente

    optimizer = Adam(learning_rate=0.1)
    model.compile(optimizer=optimizer, loss=custom_loss2, metrics = [custom_score2] )
    return model


def createLSTMlargeModel(l1Nodes, l2Nodes, l3Nodes, l4Nodes, d1dense , d2dense, input_shape): 
    lstm1 = LSTM(l1Nodes, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d1 = Dropout(0.2)
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d2 = Dropout(0.2)
    lstm3 = LSTM(l3Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d3 = Dropout(0.2)
    lstm4 = LSTM(l4Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d4 = Dropout(0.2)

    flatten = Flatten()

    dense1 = Dense(d1dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    output_dense = Dense(1,activation='relu', kernel_regularizer=regularizers.l2(0.1))

    layers = [lstm1,d1, lstm2,d2, lstm3, d3, lstm4, d4,flatten,  dense1, dense2, output_dense]
   
    optimizer = Adam(learning_rate=0.1)
    model = Sequential(layers)
    model.compile(optimizer=optimizer, loss=custom_loss2, metrics = [custom_score2])
    return model


In [9]:
LSTMlargeModel= createLSTMlargeModel(128, 64, 32, 16, 64, 32, (30,14))
LSTMsmallModel = createLSTMsmallModel(60, 30, 30, 15, (30, 14)) 

2023-09-18 15:18:29.080472: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-18 15:18:29.080528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-09-18 15:18:29.081143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
LSTMsmallModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 60)            18000     
                                                                 
 lstm_5 (LSTM)               (None, 30, 30)            10920     
                                                                 
 flatten_1 (Flatten)         (None, 900)               0         
                                                                 
 dense_3 (Dense)             (None, 30)                27030     
                                                                 
 dense_4 (Dense)             (None, 15)                465       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 56,431
Trainable params: 56,431
Non-trai

In [11]:
LSTMlargeModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           73216     
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            49408     
                                                                 
 dropout_1 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 32)            12416     
                                                                 
 dropout_2 (Dropout)         (None, 30, 32)            0         
                                                                 
 lstm_3 (LSTM)               (None, 30, 16)            3

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model, save_model

EPOCHS = 50 #100 / 80 / 150 

path_small = './models/LSTMsmall/FD0001'
path_large = './models/LSTMlarge/FD0001'


model_checkpoint = ModelCheckpoint(
    path_small,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)


early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)



In [13]:
history_small = LSTMsmallModel.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs = EPOCHS,
    batch_size = 256,
    use_multiprocessing =True, 
    callbacks=[model_checkpoint, early_stopping])

Epoch 1/50
77/78 [============================>.] - ETA: 0s - loss: 8312.9922 - custom_score2: 580985028608.0000
Epoch 1: val_loss improved from inf to 6955.92529, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


78/78 [==============================] - 15s 142ms/step - loss: 8306.5391 - custom_score2: 573536468992.0000 - val_loss: 6955.9253 - val_custom_score2: 16503478.0000
Epoch 2/50
77/78 [============================>.] - ETA: 0s - loss: 6419.3120 - custom_score2: 3036950.0000
Epoch 2: val_loss improved from 6955.92529 to 6531.10693, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


78/78 [==============================] - 8s 107ms/step - loss: 3200.7485 - custom_score2: 96748683264.0000 - val_loss: 3040.6775 - val_custom_score2: 51461988352.0000
Epoch 13/50
77/78 [============================>.] - ETA: 0s - loss: 3199.0132 - custom_score2: 198784909312.0000
Epoch 13: val_loss did not improve from 3040.67749
78/78 [==============================] - 4s 46ms/step - loss: 3200.4565 - custom_score2: 196236394496.0000 - val_loss: 3250.3374 - val_custom_score2: 175381408.0000
Epoch 14/50
77/78 [============================>.] - ETA: 0s - loss: 3104.6436 - custom_score2: 344434180096.0000
Epoch 14: val_loss did not improve from 3040.67749
78/78 [==============================] - 3s 43ms/step - loss: 3105.3865 - custom_score2: 340018364416.0000 - val_loss: 3404.3240 - val_custom_score2: 25366749184.0000
Epoch 15/50
77/78 [============================>.] - ETA: 0s - loss: 3075.3479 - custom_score2: 405437120512.0000
Epoch 15: val_loss improved from 3040.67749 to 2912.22192

INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


78/78 [==============================] - 8s 104ms/step - loss: 3075.6924 - custom_score2: 401564172288.0000 - val_loss: 2912.2219 - val_custom_score2: 24419569664.0000
Epoch 16/50
77/78 [============================>.] - ETA: 0s - loss: 3134.6252 - custom_score2: 154119176192.0000
Epoch 16: val_loss did not improve from 2912.22192
78/78 [==============================] - 4s 48ms/step - loss: 3138.0825 - custom_score2: 152338137088.0000 - val_loss: 4048.2710 - val_custom_score2: 554432790528.0000
Epoch 17/50
77/78 [============================>.] - ETA: 0s - loss: 3377.7344 - custom_score2: 49563799552.0000
Epoch 17: val_loss did not improve from 2912.22192
78/78 [==============================] - 4s 49ms/step - loss: 3377.9299 - custom_score2: 48928366592.0000 - val_loss: 3115.7849 - val_custom_score2: 215022800.0000
Epoch 18/50
77/78 [============================>.] - ETA: 0s - loss: 3438.7170 - custom_score2: 11418422272.0000
Epoch 18: val_loss did not improve from 2912.22192
78/78 [

In [14]:
model_checkpoint = ModelCheckpoint(
    path_large,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)
history_large = LSTMlargeModel.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs = 100,
    batch_size = 32,
    use_multiprocessing =True, 
    callbacks=[model_checkpoint, early_stopping])

Epoch 1/100
618/618 [==============================] - ETA: 0s - loss: 6739.1763 - custom_score2: 64832245760.0000
Epoch 1: val_loss improved from inf to 6207.61963, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 52s 73ms/step - loss: 6739.1763 - custom_score2: 64832245760.0000 - val_loss: 6207.6196 - val_custom_score2: 456881.8125
Epoch 2/100
617/618 [============================>.] - ETA: 0s - loss: 6141.1606 - custom_score2: 531571.8750
Epoch 2: val_loss improved from 6207.61963 to 6172.57764, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 46s 74ms/step - loss: 6018.3311 - custom_score2: 320490.3125 - val_loss: 6115.0645 - val_custom_score2: 308753.0938
Epoch 11/100
618/618 [==============================] - ETA: 0s - loss: 6010.9487 - custom_score2: 318703.1875
Epoch 11: val_loss did not improve from 6115.06445
618/618 [==============================] - 35s 57ms/step - loss: 6010.9487 - custom_score2: 318703.1875 - val_loss: 6146.5454 - val_custom_score2: 171605.2188
Epoch 12/100
617/618 [============================>.] - ETA: 0s - loss: 6022.0967 - custom_score2: 322979.0625
Epoch 12: val_loss did not improve from 6115.06445
618/618 [==============================] - 34s 56ms/step - loss: 6021.7397 - custom_score2: 322460.7188 - val_loss: 6173.5015 - val_custom_score2: 136936.1406
Epoch 13/100
618/618 [==============================] - ETA: 0s - loss: 6018.5347 - custom_score2: 315124.9062
Epoch 13: val_loss improved from 6115.06445 to 6115.06201, saving model to ./models/LSTM

INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 47s 77ms/step - loss: 6018.5347 - custom_score2: 315124.9062 - val_loss: 6115.0620 - val_custom_score2: 326739.5000
Epoch 14/100
618/618 [==============================] - ETA: 0s - loss: 6017.0645 - custom_score2: 309243.8125
Epoch 14: val_loss did not improve from 6115.06201
618/618 [==============================] - 39s 62ms/step - loss: 6017.0645 - custom_score2: 309243.8125 - val_loss: 6115.5898 - val_custom_score2: 296193.4375
Epoch 15/100
618/618 [==============================] - ETA: 0s - loss: 6016.1807 - custom_score2: 315945.7188
Epoch 15: val_loss did not improve from 6115.06201
618/618 [==============================] - 46s 74ms/step - loss: 6016.1807 - custom_score2: 315945.7188 - val_loss: 6115.2422 - val_custom_score2: 330412.6562
Epoch 16/100
618/618 [==============================] - ETA: 0s - loss: 6019.8711 - custom_score2: 319661.3750
Epoch 16: val_loss did not improve from 6115.06201
618/618 [============================